In [8]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Dropdown, fixed
from IPython.display import display
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go

In [9]:
# Path to go up one level from the current folder to the parent folder
parent_path = os.path.join(os.getcwd(), '..')

# Path to the target data folder
data_folder_path = os.path.join(parent_path, 'Data')

In [10]:
menu_df = pd.read_csv(os.path.join(data_folder_path, 'menu_analysis.csv'))
customers_df = pd.read_csv(os.path.join(data_folder_path, 'customers.csv'))
stores_df = pd.read_csv(os.path.join(data_folder_path, 'stores.csv'))
structure_df = pd.read_csv(os.path.join(data_folder_path, 'structure.csv'))
size_df = pd.read_csv(os.path.join(data_folder_path, 'order_size.csv'))
transactions_df = pd.read_csv(
        os.path.join(data_folder_path, "transactions.csv"),
        parse_dates=['Transaction_Date']  # Modify as needed if format issues arise
    )

In [21]:
stores_df.head()

,Store_Name,Store_Number,Address,City,State,Zip
0,Store_1,STR_1,2377 Mcdonald Pass Suite 118,Plymouth,MN,24566
1,Store_2,STR_2,161 Stephen Mission Suite 396,St. Paul,MN,19590
2,Store_3,STR_3,9285 Lyons Valleys Apt. 195,Bloomington,MN,92197
3,Store_4,STR_4,6185 Holden Highway,Woodbury,MN,76518
4,Store_5,STR_5,72128 Moore Wall,Duluth,MN,72743


### Modification 1

In [11]:
# Merge transactions with menu data
merged_df = pd.merge(transactions_df, menu_df[['Item_Number', 'Group', 'Price']], on='Item_Number')

In [13]:
merged_df.tail()

,Order_Number,Transaction_Date,Transaction_Time,Store_Number,Customer_ID,Item_Number,Group,Price
267787,60675,2023-07-21,11:18:23,STR_1,CUST9344,Item_36,Beverage,3.0
267788,13694,2024-06-08,22:17:44,STR_10,CUST8980,Item_36,Beverage,3.0
267789,4378,2024-06-18,08:51:14,STR_2,CUST5308,Item_36,Beverage,3.0
267790,107363,2023-11-04,13:08:08,STR_1,CUST5655,Item_36,Beverage,3.0
267791,72786,2023-06-28,19:34:47,STR_4,CUST5845,Item_36,Beverage,3.0


### Modification 2

In [17]:
total_transaction_df = merged_df.groupby(['Order_Number', 'Transaction_Date','Store_Number', 'Customer_ID'])['Price'].sum().reset_index()

In [18]:
total_transaction_df.head()

,Order_Number,Transaction_Date,Store_Number,Customer_ID,Price
0,1000,2024-01-09,STR_4,CUST6467,11.5
1,1001,2024-03-19,STR_9,CUST7534,27.0
2,1002,2023-07-14,STR_9,CUST5791,8.0
3,1003,2023-07-07,STR_8,CUST2406,6.5
4,1007,2023-06-21,STR_7,CUST6394,31.5


### Modification 3

In [19]:
# Calculate revenue by group, store, and date
revenue_by_group = merged_df.groupby(['Transaction_Date', 'Store_Number'])['Price'].sum().reset_index()

In [20]:
revenue_by_group.head()

,Transaction_Date,Store_Number,Price
0,2023-06-21,STR_1,331.5
1,2023-06-21,STR_10,344.0
2,2023-06-21,STR_2,507.0
3,2023-06-21,STR_3,306.0
4,2023-06-21,STR_4,403.5


#### Modification 4

In [23]:
avg_transaction = merged_df.groupby(['Transaction_Date', 'Store_Number'])['Price'].mean().reset_index()

In [24]:
avg_transaction.head()

,Transaction_Date,Store_Number,Price
0,2023-06-21,STR_1,5.525000
1,2023-06-21,STR_10,5.931034
2,2023-06-21,STR_2,5.895349
3,2023-06-21,STR_3,5.773585
4,2023-06-21,STR_4,5.683099


# 6. Revenue Distribution by Menu Group Chart

In [26]:
# Create the plot
fig = go.Figure()

for store in avg_transaction['Store_Number'].unique():
    store_data = avg_transaction[avg_transaction['Store_Number'] == store]
    fig.add_trace(go.Scatter(
        x=store_data['Transaction_Date'],
        y=store_data['Price'],
        mode='lines',
        name=f'Store {store}',
        hovertemplate='<b>Store %{name}</b><br>Date: %{x|%Y-%m-%d}<br>Avg Transaction: $%{y:.2f}<extra></extra>'
    ))

fig.update_layout(
    title='Average Transaction Value by Store Over Time',
    xaxis_title='Transaction_Date',
    yaxis_title='Average Transaction Value ($)',
    legend_title='Store',
    hovermode='closest'
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Add annotations for overall average lines
overall_avg = merged_df.groupby('Store_Number')['Price'].mean()
for store, avg in overall_avg.items():
    fig.add_shape(
        type="line",
        x0=transactions_df['Transaction_Date'].min(),
        x1=transactions_df['Transaction_Date'].max(),
        y0=avg,
        y1=avg,
        line=dict(color="rgba(0,0,0,0.3)", width=2, dash="dash"),
    )
    fig.add_annotation(
        x=transactions_df['Transaction_Date'].max(),
        y=avg,
        text=f"Store {store} Avg: ${avg:.2f}",
        showarrow=False,
        yshift=10
    )

fig.show()